In [3]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
from datetime import date
from datetime import timedelta

Выбираем 9 финансовых инструментов для анализа: "AMZN", "GOOG", "TSLA", "MBG.DE", "DTG.DE", "VOW.DE", "BTC-USD", "ETH-USD", "SOL-USD".
Интервал - 1 день, период - от 01 февраля 2022 до вчерашнего дня от дня загрузки.
Скачиваем данные с YAHOO и складываем их в словарь df_dict.
В дальнейшем можно сделать словарь словарей для хранения технических индикаторов и других данный по инструменту в 
одном словаре.

In [30]:
df_dict = {}
tickers = ["AMZN", "GOOG", "TSLA", "MBG.DE", "DTG.DE", "VOW.DE", "BTC-USD", "ETH-USD", "SOL-USD"]
yesterday = date.today() - timedelta(days = 1)

for ticker in tickers:

    df_dict[ticker] = yf.download(ticker, start='2022-02-01', end=yesterday, interval='1d').drop(columns=['Adj Close'])
    df_dict[ticker].index = pd.to_datetime(df_dict[ticker].index)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [21]:
df_dict.keys()

dict_keys(['AMZN', 'GOOG', 'TSLA', 'MBG.DE', 'DTG.DE', 'VOW.DE', 'BTC-USD', 'ETH-USD', 'SOL-USD'])

In [22]:
df_dict['AMZN'].head()

Price,Close,High,Low,Open,Volume
Ticker,AMZN,AMZN,AMZN,AMZN,AMZN
Date,,,,,
2022-02-01 00:00:00+00:00,151.193497,151.707993,147.627502,150.000000,59220000
2022-02-02 00:00:00+00:00,150.612503,155.074997,148.863495,155.050507,87330000
2022-02-03 00:00:00+00:00,138.845505,144.247498,138.332993,141.737503,225532000
2022-02-04 00:00:00+00:00,157.639496,161.199997,150.608002,155.606506,253456000
2022-02-07 00:00:00+00:00,157.935501,162.690994,157.250000,158.520004,102624000


In [23]:
df_dict['AMZN'].columns

MultiIndex([( 'Close', 'AMZN'),
            (  'High', 'AMZN'),
            (   'Low', 'AMZN'),
            (  'Open', 'AMZN'),
            ('Volume', 'AMZN')],
           names=['Price', 'Ticker'])

In [31]:
# Убираем тикеры из мультииндекса
for ticker in tickers:

    df_dict[ticker].columns = [f'{Price}' for Price, Ticker in df_dict[ticker].columns]

In [25]:
# Проверяем один датасет на пропуски и видим, что YAHOO хранит полные данные.
df_dict['AMZN'].isna().sum()

Close     0
High      0
Low       0
Open      0
Volume    0
dtype: int64

In [26]:
# Смотрим статистику. Анализ на наличие выбросов во временных рядах обычно делается на основе остаточного
# стандартного отклонения для регрессии или другой предсказательной модели. 
# С другой стороны, все эти данные взяты из реальных торгов, поэтому удалять часть данных как выбросы может быть неразумно.     
df_dict['AMZN'].describe()

,Close,High,Low,Open,Volume
count,713.000000,713.000000,713.000000,713.000000,7.130000e+02
mean,141.536335,143.356960,139.610421,141.535204,5.874599e+07
std,33.361580,33.416513,33.247010,33.378028,2.857548e+07
min,81.820000,83.480003,81.430000,82.800003,2.164740e+07
25%,113.779999,115.830002,112.059998,113.830002,4.094830e+07
50%,138.070007,139.960007,136.750000,138.699997,5.229950e+07
75%,174.119995,175.850006,172.539993,174.220001,6.848800e+07
max,214.100006,215.899994,210.880005,214.160004,2.726620e+08


In [28]:
# Нарисуем один свечной график.
fig = go.Figure(data=[go.Candlestick(x=df_dict['AMZN'].index,
                       open=df_dict['AMZN'].Open, high=df_dict['AMZN'].High,
                       low=df_dict['AMZN'].Low, close=df_dict['AMZN'].Close)])

fig.show()

Нарисуем для сравнения все 9 графиков отношения цены закрытия к средней цене закрытия за наш период наблюдений.
Видим рост и расхождение волатильности у разных инструментов с февраля 2024.

In [49]:
fig = go.Figure()

# Добавляем графики
for ticker in tickers:
    price_mean = df_dict[ticker].Close.mean()
    fig.add_trace(go.Scatter(x=df_dict[ticker].index, y=df_dict[ticker].Close/price_mean,
                             mode='lines', name=ticker,
                             line=dict(width=1)))

# Настройка осей и названий
fig.update_layout(title='Interval volatility',
                  xaxis_title='Date',
                  yaxis_title='Close Price/price_mean',
                  template='plotly_dark')

# Отображаем график
fig.show()